In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
CEREBRAS_API_KEY=os.getenv('CEREBRAS_API_KEY')
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING=os.getenv('LANGSMITH_TRACING')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_PROJECT=os.getenv('LANGSMITH_PROJECT')
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')

In [87]:
os.environ['CEREBRAS_API_KEY'] = CEREBRAS_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['LANGSMITH_API_KEY'] = LANGSMITH_API_KEY
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGSMITH_PROJECT'] = LANGSMITH_PROJECT
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [88]:
from langchain_cerebras import ChatCerebras

llm = ChatCerebras(
    model="llama-3.3-70b",
)

In [90]:
while True:
    question = input("enter a prompt, to quit press q: ")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye!")
        break

enter a prompt, to quit press q:  quit


goodbye!


Merge with artificial intelligent systems.


enter a prompt, to quit press q:  quit


goodbye!


In [91]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [92]:
store = {}

In [93]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [94]:
config = {"configurable": {"session_id": "firstchat"}}

In [95]:
model_with_memory = RunnableWithMessageHistory(llm, get_session_history)

In [96]:
model_with_memory.invoke(("Hi, i am Ochi Ojie"), config=config).content

"Hello Ochi Ojie, it's nice to meet you. Is there something I can help you with or would you like to chat?"

In [97]:
model_with_memory.invoke(("what is my name?"), config=config).content

'Your name is Ochi Ojie.'

In [98]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, i am Ochi Ojie', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Ochi Ojie, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 43, 'total_tokens': 72, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_4a36a9b9a8b8ef701032', 'id': 'chatcmpl-b4fb5ac4-60ed-4b0a-b1c5-01475e645b97', 'finish_reason': 'stop', 'logprobs': None}, id='run-65522da4-f03a-4d34-8ee6-be08b03e41ff-0', usage_metadata={'input_tokens': 43, 'output_tokens': 29, 'total_tokens': 72, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Ochi Ojie.', additional_kwargs={'refusal': None},

# RAG with LCEL

In [99]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [100]:
# from sentence_transformers import SentenceTransformer
# embedding = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

from langchain_huggingface import HuggingFaceEmbeddings

# Wrapping the SentenceTransformer with LangChain-compatible embedding
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

In [101]:
## Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls = TextLoader)
docs = loader.load()

### Creating chunks using recursive character splitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

### Creating retriever using vector DB

db = Chroma.from_documents(new_docs, embedding)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [102]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [103]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [104]:
question = "what is minoxidil ? can you tell me how it's useful to me?"
print(retrieval_chain.invoke(question))

Minoxidil is a topical medication primarily used. Unfortunately, the provided context does not specify its uses or benefits. However, it can be inferred that Minoxidil is used for a specific medical purpose, but its exact utility to you would depend on the condition it is intended to treat, which is not mentioned in the given context.


In [105]:
question = "what is minoxidil FDA-approved for ?"
print(retrieval_chain.invoke(question))

The provided context does not specify what Minoxidil is FDA-approved for. It only mentions that Minoxidil is FDA-approved, but the purpose or condition it's approved for is not stated.


In [106]:
question = "what is llama3 ? can you highlight 3 important points"
print(retrieval_chain.invoke(question))

Based on the given context, here are 3 important points about Llama 3:

1. **Llama 3 has an 8B parameter version**: The context mentions that there is an 8B parameter version of Llama 3, indicating that it is a large language model with a significant number of parameters.
2. **Llama 3 was released by Meta**: The context states that "Alongside the release of Llama 3, Meta added", suggesting that Meta is the organization responsible for releasing Llama 3.
3. **Llama 3 is a language model**: Although the context does not explicitly define what Llama 3 is, based on the mention of an 8B parameter version, it can be inferred that Llama 3 is a type of large language model, likely used for natural language processing tasks.


# Tools and Agents

In [110]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [111]:
api_wrapper = WikipediaAPIWrapper()

In [112]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [113]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [114]:
print(tool.run({'query':'langchain'}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information. Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves la

In [118]:
from langchain_community.tools import YouTubeSearchTool

In [119]:
youtube_tool = YouTubeSearchTool()

In [120]:
youtube_tool.name

'youtube_search'

In [121]:
youtube_tool.run("sunny savita")

"['https://www.youtube.com/watch?v=3ZauAhbTyBE&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=ktjJAxaX8rc&pp=ygUMc3Vubnkgc2F2aXRh']"

In [122]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [123]:
search_tool = TavilySearchResults()

In [124]:
search_tool.invoke("what is happening between EU and Russia concerning energy?")

[{'title': 'EU plans to end Russian gas imports by end of 2027 - BBC',
  'url': 'https://www.bbc.com/news/articles/c04547pr9vqo',
  'content': 'In response to the plans, Kremlin spokesperson Dmitry Peskov told the Reuters news agency on Tuesday that Europe was "shooting itself in the foot".\nThe EU vowed to end its energy relations with Russia after Moscow launched its full-scale invasion of Ukraine in 2022.\nA set of legislative proposals will be tabled in June asking all EU member states to make "national plans" to phase out Russian gas, nuclear fuel and oil imports, according to the European Commission.\nEuropean Union [...] The EU hopes to move away from Russian gas by "enhancing energy efficiency, accelerating the deployment of renewable energy and diversifying supplies", the European Commission said.\nWhile EU dependence on Russian oil, gas and nuclear fuels has decreased significantly since Putin\'s invasion of Ukraine, the bloc still imports a significant quantity of its energy

In [126]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [127]:
tool = load_tools(['wikipedia'], llm=llm)

In [129]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/var/folders/_f/t1w94w1n759_mq486wwt1z9m0000gn/T/ipykernel_31933/2646369436.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [130]:
agent.run("WHat is the current fintech market size in nigeria")

/var/folders/_f/t1w94w1n759_mq486wwt1z9m0000gn/T/ipykernel_31933/2368474919.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("WHat is the current fintech market size in nigeria")




> Entering new AgentExecutor chain...
Thought: To find the current fintech market size in Nigeria, I need to access up-to-date information on the topic. Since the wikipedia tool is available, I can use it to search for relevant information. However, I should note that Wikipedia may not always have the most current data, so the information might be approximate or based on the latest available reports.

Action: wikipedia
Action Input: fintech market size in Nigeria
Observation: Page: Economy of Nigeria
Summary: The economy of Nigeria is a middle-income, mixed economy and emerging market with expanding manufacturing, financial, service, communications, technology, and entertainment sectors. It is ranked as the 53rd-largest economy in the world in terms of nominal GDP, the sixth largest in Africa and the 27th-largest in terms of purchasing power parity.
The country's re-emergent manufacturing sector became the largest on the continent in 2013, and it produces a large proportion of goods 

'The current fintech market size in Nigeria is not available in the provided data.'

In [131]:
agent.run("WHat is the current GDP of nigeria")



> Entering new AgentExecutor chain...
Thought: To find the current GDP of Nigeria, I should search for the most recent data available on Wikipedia, which may have information on Nigeria's economy, including its GDP.

Action: wikipedia
Action Input: Nigeria GDP
Observation: Page: Economy of Nigeria
Summary: The economy of Nigeria is a middle-income, mixed economy and emerging market with expanding manufacturing, financial, service, communications, technology, and entertainment sectors. It is ranked as the 53rd-largest economy in the world in terms of nominal GDP, the sixth largest in Africa and the 27th-largest in terms of purchasing power parity.
The country's re-emergent manufacturing sector became the largest on the continent in 2013, and it produces a large proportion of goods and services for the region of West Africa. Nigeria's debt-to-GDP ratio was 36.63% in 2021 according to the IMF.
Although oil revenues contributed 2/3 of state revenues, oil only contributes about 9% to the 

'The current GDP of Nigeria is not available in the search results.'